# RAG normativa ARN


## Descarga y ejecuta Llama 3.1 de 8B vía ollama

Based on https://cubed.run/blog/running-open-ai-models-for-free-in-under-10-minutes-with-a-google-colab-and-no-extra-accounts-yes-4d38b59f0153

In [1]:
#Installs Ollama
!curl -fsSL https://ollama.com/install.sh | sh
# Runs Ollama service in the background
!ollama serve &>/dev/null&
# Downloads Llama 3.1 model (8b parameters)
!ollama pull llama3.1:8b

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏ 582 KB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕▏  35 

## Extrae texto desde PDF

Podría tomar el texto extraido previamente.

In [5]:
# generar texto desde PDF

# if where pypdf is  insalled
# !pip install pypdf

!pip install langchain_community
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://www.argentina.gob.ar/sites/default/files/0-0-1_r2.pdf")
#loader = PyPDFLoader("https://arxiv.org/pdf/2405.07437.pdf")
docs = loader.load()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


## Embeddings

### Genera embeddings

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

### Guarda vectores (embeddings) en base de datos Chroma

In [ ]:
! pip install chromadb
! ollama pull all-minilm  #modelo para el embedding

In [28]:
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

emb = OllamaEmbeddings(model="all-minilm")
vectorstore = Chroma.from_documents(documents=splits, embedding=emb)

### Verificación operación BD | Búsqueda por similaridad

In [74]:
# Perform a similarity search on the Chroma vector store
query = "Como funciona un rector nuclear"
results = vectorstore.similarity_search(query=query, k = 5)  # k is the number of top results you want to retrieve
results

[Document(metadata={'page': 3, 'source': 'https://www.argentina.gob.ar/sites/default/files/0-0-1_r2.pdf'}, page_content='te. \n23. Todo cambio en la organización de la Entidad Responsable que pueda afectar su capaci-\ndad para afrontar las responsabilidades que se han definido anteriormente requerirá, para que \nlas licencias de las instalaciones Clase I afectadas mantengan su vigencia, una aprobación \nformal de la Autoridad Regulatoria.  \n24. La Entidad Responsable designará al Responsable Primario de una instalación Clase I, \nquien será el responsable directo de la seguridad radiológica y nuclear, así como del cumpli-\nmiento de las licencias, normas y requerimientos. La Entidad Responsable prestará al Respon-\nsable Primario todo el apoyo que necesite y realizará una supervisión adecuada para \ngarantizar que la instalación Clase I se opere en condiciones seguras y conforme a los térmi-\nnos de la licencia de operación. \n25. Previamente a la solicitud de una licencia, la Entidad

### Retriever


In [36]:
retriever = vectorstore.as_retriever()

## Compendio de documentos

In [38]:
def prepare_documents(results):
  documents = ""
  for i, result in enumerate(results):
    documents+= f'Doc nr. {i} \n Metadata: {result.metadata} \n Content: {result.page_content} \n'
  return documents

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

## Chat



### Plantilla para la conversación

In [57]:
template = f"""
Sos un experto en seguridad nuclear  dispuesto a responder preguntas sobre las normas y guias emitidas por la Autoridad Regulatoria Nuclear de Argentina.Tu tarea es responder la pregunta provista abajo.
Tambien se proveen una serie de documentos que un retrieval engine considero mas similares a la pregunta.
Provees respuestas concisas siguiendo la lógica "pensemos paso a paso".
Pregunta: {query}
Documentos: {prepare_documents(results)}
"""


# from langchain_core.prompts.chat import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_template(template)
from langchain_core.prompts.chat import PromptTemplate
prompt = PromptTemplate(
    template = template,
    input_variables=["query", "documents"]
    )


In [60]:
from langchain_core.prompts.chat import PromptTemplate
prompt = PromptTemplate(
  template=
  """
  Sos un experto en seguridad nuclear  dispuesto a responder preguntas sobre las normas y guias emitidas por la Autoridad Regulatoria Nuclear de Argentina.Tu tarea es responder la pregunta provista abajo.
  Tambien se proveen una serie de documentos que un retrieval engine considero mas similares a la pregunta.
  Provees respuestas concisas siguiendo la lógica "pensemos paso a paso".

  Pregunta: {question}
  Documentos: {documents}
  Respuesta:
  """,
  input_variables=["question", "documents"],
)

Define la cadena del rag

Fuente: https://www.datacamp.com/tutorial/llama-3-1-rag



Inicializa el LLM

In [ ]:
# !pip install langchain-ollama
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.1:8b",
    temperature = 0,
)

In [72]:
from langchain_core.output_parsers import StrOutputParser
rag_chain = prompt | llm | StrOutputParser()

In [63]:
# Define the RAG application class
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain
    def run(self, question):
        # Retrieve relevant documents
        documents = self.retriever.invoke(question)
        # Extract content from retrieved documents
        doc_texts = "\\n".join([doc.page_content for doc in documents])
        # Get the answer from the language model
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

In [65]:
! ollama pull llama3.1:8b

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 1a4c3c319823... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [73]:
# Initialize the RAG application
rag_application = RAGApplication(retriever, rag_chain)
# Example usage
question = "Definir instalación minera"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Definir instalación minera
Answer: Pensemos paso a paso:

La pregunta es: "Definir instalación minera"

Según el documento proporcionado, la definición de "Instalación Minero Fabril" es:

"Instalación destinada a la extracción y concentración de minerales que contienen radionucleidos de la serie del uranio o de la serie del torio, a los efectos de producir concentrado de uranio o de torio, y que puede incluir el sitio de disposición final de residuos radiactivos provenientes de esa producción."

Pero, en el contexto de la pregunta, es más probable que se refiera a "Instalación Minera" en general, sin especificar si es Fabril o no.

En ese caso, podemos considerar que una Instalación Minera es cualquier instalación destinada a la extracción y procesamiento de minerales, incluyendo la posible generación de residuos radiactivos.

Por tanto, la respuesta sería:

"Una instalación minera es cualquier instalación destinada a la extracción y procesamiento de minerales, que puede gene

## Definición de la consulta al RAG

In [40]:
# Esto no se usa.
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [69]:
# Invoca el modelo asegurándote de pasar los argumentos correctos
query = "cuales son los criterios para otorgar una licencia para una instalación de clase I segun la norma AR 0.0.1?"
formatted_documents = prepare_documents(results)

# Llamada corregida con los argumentos requeridos
rag_chain.invoke({"query": query, "documents": formatted_documents})

KeyError: "Input to PromptTemplate is missing variables {'question'}.  Expected: ['documents', 'question'] Received: ['query', 'documents']\nNote: if you intended {question} to be part of the string and not a variable, please escape it with double curly braces like: '{{question}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT"